<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN04/4.2-Gradient_Descent.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
%pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Gradient Descent


In diesem Abschnitt werden wir die grundlegenden Konzepte des *Gradientenabstiegs* vorstellen.
- Obwohl er beim Deep Learning nur selten direkt verwendet wird, ist ein Verständnis des Gradientenabstiegs der Schlüssel zum Verständnis der stochastischen Gradientenabstiegsalgorithmen. 
- So kann das Optimierungsproblem beispielsweise aufgrund einer zu hohen Lernrate divergieren. Dieses Phänomen kann bereits beim Gradientenabstieg beobachtet werden. Ebenso ist die Vorkonditionierung eine gängige Technik beim Gradientenabstieg, die sich auch auf fortgeschrittenere Algorithmen überträgt.

Lassen Sie uns mit einem einfachen Spezialfall beginnen.



## Eindimensionaler Gradientenabstieg

Der eindimensionale Gradientenabstieg ist ein hervorragendes Beispiel, um zu erklären, warum der Gradientenabstiegsalgorithmus den Wert der Zielfunktion verringern kann. Betrachten wir eine kontinuierlich differenzierbare reellwertige Funktion $f: \mathbb{R} \rightarrow \mathbb{R}$. Unter Verwendung einer Taylor-Erweiterung erhalten wir

$$f(x + \epsilon) = f(x) + \epsilon f'(x) + \mathcal{O}(\epsilon^2). \tag{1}$$


Das heisst, bei der Approximation erster Ordnung ist $f(x+\epsilon)$ durch den Funktionswert $f(x)$ und die erste Ableitung $f'(x)$ nach $x$ gegeben. Es ist nicht unvernünftig anzunehmen, dass für kleine $\epsilon$ eine Bewegung in Richtung des negativen Gradienten den Funktionswert $f$ verringert. Der Einfachheit halber wählen wir eine feste Schrittweite $\eta > 0$ und wählen $\epsilon = -\eta f'(x)$. Wenn wir dies in die obige Taylorentwicklung einsetzen, erhalten wir

$$f(x - \eta f'(x)) = f(x) - \eta f'^2(x) + \mathcal{O}(\eta f'^2(x)). \tag{2}$$

Wenn die Ableitung $f'(x) \neq 0$ nicht verschwindet, kommen wir weiter, da $\eta f'^2(x)>0$. Außerdem können wir $\eta$ immer so klein wählen, dass die Terme höherer Ordnung irrelevant werden. Wir kommen also zu

$$f(x - \eta f'(x)) \lessapprox f(x). \tag{3}$$

Das bedeutet, dass, wenn wir

$$x \leftarrow x - \eta f'(x)$$

verwenden, um $x$ zu iterieren, kann der Wert der Funktion $f(x)$ sinken. Daher wählen wir beim Gradientenabstieg zunächst einen Anfangswert $x$ und eine Konstante $\eta > 0$ und iterieren damit kontinuierlich $x$ bis zum Erreichen der Stop-Bedingung, z.B. wenn die Grösse des Gradienten $|f'(x)|$ klein genug ist oder die Anzahl der Iterationen einen bestimmten Wert erreicht hat.



Der Einfachheit halber wählen wir die Zielfunktion $f(x)=x^2$, um zu veranschaulichen, wie man den Gradientenabstieg implementiert. Obwohl wir wissen, dass $x=0$ die Lösung zur Minimierung von $f(x)$ ist, verwenden wir diese einfache Funktion, um zu beobachten, wie sich $x$ verändert.

In [ ]:
# !pip install requests
# !pip install autograd
# !git clone https://github.com/dsgiitr/d2l-pytorch.git
# Homepage
# https://d2l.ai/

# %pip install d2l # geht nur mit 3.11.9 python version

In [ ]:
# pip install -U d2l
# !pip install requests
%matplotlib inline
import sys
import math
import numpy as np
import torch
import d2l

sys.path.insert(0, "..")


In [ ]:
def f(x):
    return x**2  # objective function


def gradf(x):
    return 2 * x  # its derivative


Als nächstes verwenden wir $x=10$ als Anfangswert und nehmen $\eta=0.2$ an. Wenn wir den Gradientenabstieg verwenden, um $x$ 10 Mal zu iterieren, können wir sehen, dass sich der Wert von $x$ schließlich der optimalen Lösung nähert.


In [ ]:
def gd(eta):
    x = 10
    results = [x]
    for i in range(10):
        x -= eta * gradf(x)
        results.append(x)
    print("epoch 10, x:", x)
    return results


res = gd(0.2)


Der Fortschritt der Optimierung über $x$ kann wie folgt aufgezeichnet werden.

In [ ]:
def show_trace(res):
    n = max(abs(min(res)), abs(max(res)))
    f_line = np.arange(-n, n, 0.01)
    d2l.set_figsize((3.5, 2.5))
    d2l.plot(
        [f_line, res],
        [[f(x) for x in f_line], [f(x) for x in res]],
        "x",
        "f(x)",
        fmts=["-", "-o"],
    )


show_trace(res)


### Lernrate $\eta$

Die Lernrate $\eta$ kann vom Entwickler des Algorithmus festgelegt werden. Wenn wir eine zu kleine Lernrate verwenden, wird $x$ sehr langsam aktualisiert, was mehr Iterationen erfordert, um eine bessere Lösung zu erhalten. Um zu zeigen, was in einem solchen Fall passiert, betrachten wir den Fortschritt bei demselben Optimierungsproblem für $\eta = 0.05$. Wie wir sehen können, sind wir selbst nach 10 Schritten noch sehr weit von der optimalen Lösung entfernt.


In [ ]:
show_trace(gd(0.05))


- Umgekehrt könnte $\left|\eta f'(x)\right|$ zu gross für die Taylor-Erweiterungsformel erster Ordnung sein, wenn wir eine zu hohe Lernrate verwenden. 
- Das heisst, der Term $\mathcal{O}(\eta^2 f'^2(x))$ in Gleichung (2) könnte signifikant werden. 
- In diesem Fall können wir nicht garantieren, dass die Iteration von $x$ in der Lage ist, den Wert von $f(x)$ zu senken. 

Wenn wir z.B. die Lernrate auf $\eta=1.1$ setzen, schwimmt $x$ über die optimale Lösung $x=0$ hinaus und divergiert allmählich.


In [ ]:
show_trace(gd(1.1))


### Lokale Minima

Um zu veranschaulichen, was bei nichtkonvexen Funktionen passiert, betrachten wir den Fall $f(x) = x \cdot \cos(cx)$ für eine Konstante $c$. Diese Funktion hat unendlich viele lokale Minima. Je nach Wahl der Lernrate und je nachdem, wie gut das Problem konditioniert ist, können wir am Ende eine von vielen Lösungen erhalten. Das folgende Beispiel veranschaulicht, wie eine (unrealistisch) hohe Lernrate zu einem schlechten lokalen Minimum führt.


In [ ]:
c = 0.15 * math.pi


def f(x):
    return x * math.cos(c * x)


def gradf(x):
    return math.cos(c * x) - c * x * math.sin(c * x)


show_trace(gd(2))


## Multivariater Gradientenabstieg

Nachdem wir nun ein besseres Verständnis für den univariaten Fall haben, wollen wir die Situation betrachten, in der $\mathbf{x} = [x_1, x_2, \ldots, x_d]^\top$ ein Vektor ist. Das heisst, die Zielfunktion $f: \mathbb{R}^d \to \mathbb{R}$ bildet Vektoren in Skalare ab. Dementsprechend ist auch ihr Gradient multivariat. Er ist ein Vektor, der aus $d$ partiellen Ableitungen besteht:

$$\nabla f(\mathbf{x}) = \bigg[\frac{\partial f(\mathbf{x})}{\partial x_1}, \frac{\partial f(\mathbf{x})}{\partial x_2}, \ldots, \frac{\partial f(\mathbf{x})}{\partial x_d}\bigg]^\top.$$


Jedes partielle Ableitungselement $\partial f(\mathbf{x})/\partial x_i$ im Gradienten gibt die Änderungsrate von $f$ bei $\mathbf{x}$ in Bezug auf die Eingabe $x_i$ an. Wie zuvor im univariaten Fall können wir die entsprechende Taylor-Approximation für multivariate Funktionen verwenden, um eine Vorstellung davon zu bekommen, was wir tun sollten. Insbesondere haben wir, dass

$$f(\mathbf{x} + \boldsymbol{\epsilon}) = f(\mathbf{x}) + \mathbf{\boldsymbol{\epsilon}}^\top \nabla f(\mathbf{x}) + \mathcal{O}(\|\boldsymbol{\epsilon}\|^2).$$
:eqlabel:`gd-multi-taylor`


Mit anderen Worten, bis zu Termen zweiter Ordnung in $\boldsymbol{\epsilon}$ ist die Richtung des steilsten Abstiegs durch den negativen Gradienten $-\nabla f(\mathbf{x})$ gegeben. Wählt man eine geeignete Lernrate $\eta > 0$, so erhält man den prototypischen Gradientenabstiegsalgorithmus:

$$\mathbf{x} \leftarrow \mathbf{x} - \eta \nabla f(\mathbf{x}).$$

Um zu sehen, wie sich der Algorithmus in der Praxis verhält, konstruieren wir eine Zielfunktion $f(\mathbf{x})=x_1^2+2x_2^2$ mit einem zweidimensionalen Vektor $\mathbf{x} = [x_1, x_2]^\top$ als Eingabe und einem Skalar als Ausgabe. Der Gradient ist gegeben durch $\nabla f(\mathbf{x}) = [2x_1, 4x_2]^\top$. Wir werden die Trajektorie von $\mathbf{x}$ durch Gradientenabstieg von der Ausgangsposition $[-5, -2]$ aus betrachten. 

Zu Beginn benötigen wir zwei weitere Hilfsfunktionen. Die erste verwendet eine Aktualisierungsfunktion und wendet sie 20 Mal auf den Ausgangswert an. Die zweite Hilfsfunktion visualisiert die Trajektorie von $\mathbf{x}$.

Als nächstes betrachten wir den Verlauf der Optimierungsvariablen $\mathbf{x}$ für die Lernrate $\eta = 0.1$. Wir sehen, dass sich der Wert von $\mathbf{x}$ nach 20 Schritten seinem Minimum bei $[0, 0]$ nähert. Der Fortschritt ist recht brav, wenn auch recht langsam.

In [ ]:
def f(x1, x2):
    return x1**2 + 2 * x2**2  # objective


def gradf(x1, x2):
    return (2 * x1, 4 * x2)  # gradient


def gd(x1, x2, s1, s2):
    (g1, g2) = gradf(x1, x2)  # compute gradient
    return (x1 - eta * g1, x2 - eta * g2, 0, 0)  # update variables


eta = 0.1
d2l.show_trace_2d(f, d2l.train_2d(gd))


## Adaptive Methoden

Wie wir in sehen konnten, ist es schwierig, die Lernrate $\eta$ "genau richtig" zu wählen. Wenn wir sie zu klein wählen, machen wir kaum Fortschritte. Wenn wir sie zu gross wählen, schwankt die Lösung und im schlimmsten Fall kann sie sogar divergieren. Was wäre, wenn wir $\eta$ automatisch bestimmen oder überhaupt keine Lernrate mehr wählen müssten? 
Methoden zweiter Ordnung, die nicht nur den Wert und den Gradienten der Zielfunktion betrachten
sondern auch auf ihre *Krümmung* schauen, können in diesem Fall helfen. Diese Methoden können zwar aufgrund des Rechenaufwands nicht direkt auf Deep Learning angewendet werden, sie liefern jedoch nützliche Anhaltspunkte für die Entwicklung fortgeschrittener Optimierungsalgorithmen, die viele der wünschenswerten Eigenschaften der unten beschriebenen Algorithmen nachahmen.


### Newtonsche Methode

Bei der Überprüfung der Taylorentwicklung einer Funktion $f: \mathbb{R}^d \rightarrow \mathbb{R}$ ist es nicht nötig, nach dem ersten Term aufzuhören. Tatsächlich können wir sie schreiben als

$$f(\mathbf{x} + \boldsymbol{\epsilon}) = f(\mathbf{x}) + \boldsymbol{\epsilon}^\top \nabla f(\mathbf{x}) + \frac{1}{2} \boldsymbol{\epsilon}^\top \nabla^2 f(\mathbf{x}) \boldsymbol{\epsilon} + \mathcal{O}(\|\boldsymbol{\epsilon}\|^3). \tag{4}$$

- Um eine umständliche Notation zu vermeiden, definieren wir $\mathbf{H} \stackrel{\mathrm{def}}{=} \nabla^2 f(\mathbf{x})$ als die Hessian von $f$, die eine $d \times d$ Matrix ist. 
- Für kleine $d$ und einfache Probleme ist $\mathbf{H}$ leicht zu berechnen. Für tiefe neuronale Netze hingegen kann $\mathbf{H}$ aufgrund der Kosten für die Speicherung von $\mathcal{O}(d^2)$-Einträgen unerschwinglich gross sein. 
- Ausserdem kann es zu teuer sein, sie mittels Backpropagation zu berechnen. Für den Moment ignorieren wir solche Überlegungen und sehen uns an, welchen Algorithmus wir erhalten würden.



Schliesslich erfüllt das Minimum von $f$ die Bedingung $\nabla f = 0$. 
Nach den Regeln des Kalküls durch Ableitung von (4) und Ignorieren von Termen höherer Ordnung erhalten wir

$$\nabla f(\mathbf{x}) + \mathbf{H} \boldsymbol{\epsilon} = 0$$
und damit
$$ \boldsymbol{\epsilon} = -\mathbf{H}^{-1} \nabla f(\mathbf{x}).$$

Das heisst, wir müssen die Hessematrix $\mathbf{H}$ als Teil des Optimierungsproblems invertieren. Dies nennt man **Vorkonditionierung (preconditioning)**.



- Ein einfaches Beispiel: Für $f(x) = \frac{1}{2} x^2$ haben wir $\nabla f(x) = x$ und $\mathbf{H} = 1$. 
- Daher erhalten wir für jedes $x$ $\epsilon = -x$. Mit anderen Worten: ein *einziger* Schritt reicht aus, um perfekt zu konvergieren, ohne dass eine Anpassung erforderlich ist! 
- Leider hatten wir hier ein wenig Glück: Die Taylorentwicklung war exakt, da $f(x+\epsilon)= \frac{1}{2} x^2 + \epsilon x + \frac{1}{2} \epsilon^2$. 

Schauen wir uns an, was bei anderen Problemen passiert.
Bei einer konvexen hyperbolischen Kosinusfunktion $f(x) = \cosh(cx)$ für eine Konstante $c$ ist zu erkennen, dass
das globale Minimum bei $x=0$ nach ein paar Iterationen erreicht wird.

In [ ]:
c = 0.5


def f(x):
    return math.cosh(c * x)  # objective


def gradf(x):
    return c * math.sinh(c * x)  # derivative


def hessf(x):
    return c**2 * math.cosh(c * x)  # hessian


# hide learning rate for now
def newton(eta=1):
    x = 10
    results = [x]
    for i in range(10):
        x -= eta * gradf(x) / hessf(x)
        results.append(x)
    print("epoch 10, x:", x)
    return results


show_trace(newton())


Betrachten wir nun eine *nichtkonvexe* Funktion, wie $f(x) = x \cdot \cos(c x)$ für eine Konstante $c$. 

- Schliesslich ist zu beachten, dass wir bei der Newton-Methode durch die Hessematrix "dividieren" müssen. Das bedeutet, dass wir, wenn die zweite Ableitung *negativ* ist, in die Richtung gehen können, dass der Wert von $f$ *erhöht* wird.
- Das ist ein fataler Fehler des Algorithmus. 

Schauen wir uns an, was in der Praxis passiert.


In [ ]:
c = 0.15 * math.pi


def f(x):
    return x * math.cos(c * x)


def gradf(x):
    return math.cos(c * x) - c * x * math.sin(c * x)


def hessf(x):
    return -2 * c * math.sin(c * x) - x * c**2 * math.cos(c * x)


show_trace(newton())


Das ging spektakulär schief. Wie können wir das beheben? 
- Eine Möglichkeit wäre, die Hessian zu "reparieren", indem man stattdessen ihren absoluten Wert nimmt. 
- Eine andere Strategie besteht darin, die Lernrate zu erhöhen. 
- Das scheint den Zweck zu vereiteln, aber nicht ganz. 
- Die Informationen zweiter Ordnung erlauben es uns, vorsichtig zu sein, wenn die Krümmung groß ist, und längere Schritte zu machen, wenn die Zielfunktion flacher ist. 

Sehen wir uns an, wie das mit einer etwas kleineren Lernrate, sagen wir $\eta = 0.5$, funktioniert. Wie wir sehen können, haben wir einen recht effizienten Algorithmus.


In [ ]:
show_trace(newton(0.5))


## Zusammenfassung

* Lernraten sind wichtig. Ist sie zu gross, divergieren wir, ist sie zu klein, machen wir keine Fortschritte.
* Gradientenabstieg kann in lokalen Minima stecken bleiben.
* In hohen Dimensionen ist die Anpassung der Lernrate kompliziert.
* Vorkonditionierung kann bei der Skalenanpassung helfen.
* Die Newton-Methode ist viel schneller, wenn sie bei konvexen Problemen erst einmal richtig funktioniert.
* Hüten Sie sich davor, die Newton-Methode ohne Anpassungen für nicht-konvexe Probleme zu verwenden.



## Exercises

1. Experimentieren Sie mit verschiedenen Lernraten und Zielfunktionen für den Gradientenabstieg.
2. Implementieren Sie die **Liniensuche** zur Minimierung einer konvexen Funktion im Intervall $[a, b]$.
    - A. Benötigen Sie Ableitungen für die binäre Suche, d.h. um zu entscheiden, ob $[a, (a+b)/2]$ oder $[(a+b)/2, b]$ gewählt werden soll.
    - B. Wie schnell ist die Konvergenzrate des Algorithmus?
    - C. Implementieren Sie den Algorithmus und wenden Sie ihn auf die Minimierung von $\log (\exp(x) + \exp(-2x -3))$ an.
3. Entwerfen Sie eine auf $\mathbb{R}^2$ definierte Zielfunktion, bei der der Gradientenabstieg äußerst langsam ist. Tipp: Skalieren Sie verschiedene Koordinaten unterschiedlich.
4. Implementieren Sie die leichtgewichtige Version der *Newtonschen Methode* unter Verwendung von **Vorkonditionierung**:
    - A. Verwenden Sie die diagonale Hessematrix als Vorkonditionierer.
    - B. Verwenden Sie die absoluten Werte anstelle der tatsächlichen (möglicherweise vorzeichenbehafteten) Werte.
    - C. Wenden Sie dies auf das obige Problem an.
5. Wenden Sie den obigen Algorithmus auf eine Reihe von Zielfunktionen (konvex oder nicht) an. Was passiert, wenn man die Koordinaten um $45$ Grad dreht?

